# NBA Data Scraper ja Over/Under Deep Learning -malli

Tämän notebookin koodinpätkillä lisätään viimeisimpien NBA otteluiden tulokset ja tilastot aikaisemmin kerättyjen otteluiden kanssa samaan dataframeen, josta lasketaan Deep Learning -mallissa käytettävien riippumattomien muuttujien arvot. DL-mallissa riippuva muuttuja, jota mallin ennustaa, on tulevien otteluiden pistemäärät koti- ja vierasjoukkueille, josta saadaan arvio ottelun kokonaispistemäärästä. Projekti on muutaman vuoden vanha, minkä takia koodi ei tällä hetkellä toimi eri kirjastojen versiopäivitysten takia.

In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
import time
import feather
import fastai
from IPython.display import display

Muutama hyödyllinen funktio datafreimien tarkasteluun ja liittämiseen sekä painotettujen liikkuvien keskiarvojen laskuun.

In [2]:
def display_all(df):
    with pd.option_context("display.max_rows", 15000, "display.max_columns", 1000):
        display(df)

In [3]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

In [4]:
def weighted_ma(w):
    def g(x):
        return sum(w*x) / sum(w)
    return g

In [5]:
# Alempana lasketaan joidenkin tilastojen liikkuvia keskiarvoja eri vuosilta siten, 
# että uudemmat tilastot saavat enemmän painoarvoa. Painotus tapahtuu tässä lasketuilla painoilla

fast_ma = 3
slow_ma = 6
moving_averages =[fast_ma, slow_ma]

alpha = 0.30
fast_weights = list(reversed([(1-alpha)**n for n in range(fast_ma)]))
slow_weights = list(reversed([(1-alpha)**n for n in range(slow_ma)]))

In [6]:
PATH = "C:/Users/akibr/fastai/courses/dl1/data/NBA/"

In [7]:
bet_date = '2020-03-11'

In [8]:
# Kaikki käytettävät tilastot ja niiden perusteella malliin riippumattomiksi muuttujiksi laskettavat
# jatkuvat muuttujat sekä kategoriamuuttujat

orig_stats1 = ['PTS','FGM','FGA','FG%','3PM','3PA','3P%','FTM','FTA','FT%','OREB','DREB','REB','AST','TOV','STL',
              'BLK','PF','+/-','PPS']

orig_stats2 = ['PTS','FGM','FGA','FG%','3PM','3PA','3P%','FTM','FTA','FT%','OREB','DREB','REB','AST','TOV','STL',
              'BLK','PF','+/-','PPS','POS','OR','DR','SOS','PIE','TIE']

home_stats1 = ['homePTS','homeFGM','homeFGA','homeFG%','home3PM','home3PA','home3P%','homeFTM','homeFTA','homeFT%',
              'homeOREB','homeDREB','homeREB','homeAST','homeTOV','homeSTL','homeBLK','homePF','home+/-','homePPS']

home_stats2 = ['homePTS','homeFGM','homeFGA','homeFG%','home3PM','home3PA','home3P%','homeFTM','homeFTA','homeFT%',
              'homeOREB','homeDREB','homeREB','homeAST','homeTOV','homeSTL','homeBLK','homePF','home+/-','homePPS',
             'homePOS','homeOR','homeDR','homeSOS','homePIE','homeTIE']

away_stats1 = ['awayPTS','awayFGM','awayFGA','awayFG%','away3PM','away3PA','away3P%','awayFTM','awayFTA','awayFT%',
              'awayOREB','awayDREB','awayREB','awayAST','awayTOV','awaySTL','awayBLK','awayPF','away+/-','awayPPS']

away_stats2 = ['awayPTS','awayFGM','awayFGA','awayFG%','away3PM','away3PA','away3P%','awayFTM','awayFTA','awayFT%',
              'awayOREB','awayDREB','awayREB','awayAST','awayTOV','awaySTL','awayBLK','awayPF','away+/-','awayPPS',
             'awayPOS','awayOR','awayDR','awaySOS','awayPIE','awayTIE']

total_stats = ['totalPTS','totalFGM','totalFGA','totalFG%','total3PM','total3PA','total3P%','totalFTM','totalFTA',
               'totalFT%','totalOREB','totalDREB','totalREB','totalAST','totalTOV','totalSTL','totalBLK','totalPF',
               'total+/-','totalPPS','totalPOS','totalOR','totalDR','totalSOS','totalPIE','totalTIE']


team_map = {'Atlanta': 'ATL', 'Brooklyn': 'BKN','Boston': 'BOS','Charlotte': 'CHA','Chicago': 'CHI','Cleveland': 'CLE',
            'Dallas': 'DAL','Denver': 'DEN','Detroit': 'DET','Golden State': 'GSW','Houston': 'HOU','Indiana': 'IND',
            'LA': 'LAC','Los Angeles': 'LAL','Memphis': 'MEM','Miami': 'MIA','Milwaukee': 'MIL','Minnesota': 'MIN',
            'New Orleans': 'NOP','New York': 'NYK','Oklahoma City': 'OKC','Orlando': 'ORL','Philadelphia': 'PHI',
            'Phoenix':'PHX','Portland': 'POR','Sacramento': 'SAC','San Antonio': 'SAS','Toronto': 'TOR',
            'Utah': 'UTA','Washington': 'WAS'}


cat_vars = ['Season','homeTeam','awayTeam']

contin_vars = ['fasthomePTS','fasthome3P%','fasthome3PA','fasthome3PM','fasthomeAST','fasthomeBLK','fasthomeDREB',
               'fasthomeFG%','fasthomeFGA','fasthomeFGM','fasthomeFT%','fasthomeFTA','fasthomeFTM','fasthomeOREB', 
               'fasthomePF','fasthome+/-','fasthomePPS','fasthomeSTL','fasthomeTOV','fasthomeREB','fasthomePOS',
               'fasthomeOR','fasthomeDR','fasthomeSOS','fasthomePIE','fasthomeTIE','slowhomePTS',
               'slowhome3P%','slowhome3PA','slowhome3PM','slowhomeAST','slowhomeBLK','slowhomeDREB','slowhomeFG%',
               'slowhomeFGA','slowhomeFGM','slowhomeFT%','slowhomeFTA','slowhomeFTM','slowhomeOREB','slowhomePF',
               'slowhome+/-','slowhomePPS','slowhomeSTL','slowhomeTOV','slowhomeREB','slowhomePOS','slowhomeOR',
               'slowhomeDR','slowhomeSOS','slowhomePIE','slowhomeTIE','fastawayPTS','fastaway3P%',
               'fastaway3PA','fastaway3PM','fastawayAST','fastawayBLK','fastawayDREB','fastawayFG%','fastawayFGA',
               'fastawayFGM','fastawayFT%','fastawayFTA','fastawayFTM','fastawayOREB','fastawayPF','fastaway+/-',
               'fastawayPPS','fastawaySTL','fastawayTOV','fastawayREB','fastawayPOS','fastawayOR','fastawayDR',
               'fastawaySOS','fastawayPIE','fastawayTIE','slowawayPTS','slowaway3P%','slowaway3PA',
               'slowaway3PM','slowawayAST','slowawayBLK','slowawayDREB','slowawayFG%','slowawayFGA','slowawayFGM',
               'slowawayFT%','slowawayFTA','slowawayFTM','slowawayOREB','slowawayPF','slowaway+/-','slowawayPPS',
               'slowawaySTL','slowawayTOV','slowawayREB','slowawayPOS','slowawayOR','slowawayDR','slowawaySOS',
               'slowawayPIE','slowawayTIE','fasthometotalPTS','fasthometotal3P%','fasthometotal3PA',
               'fasthometotal3PM','fasthometotalAST','fasthometotalBLK','fasthometotalDREB','fasthometotalFG%',
               'fasthometotalFGA','fasthometotalFGM','fasthometotalFT%','fasthometotalFTA','fasthometotalFTM',
               'fasthometotalOREB','fasthometotalPF','fasthometotal+/-','fasthometotalPPS','fasthometotalSTL',
               'fasthometotalTOV','fasthometotalREB','fasthometotalPOS','fasthometotalOR','fasthometotalDR',
               'fasthometotalSOS','fasthometotalPIE','fasthometotalTIE','slowhometotalPTS','slowhometotal3P%',
               'slowhometotal3PA','slowhometotal3PM','slowhometotalAST','slowhometotalBLK','slowhometotalDREB','slowhometotalFG%',
               'slowhometotalFGA','slowhometotalFGM','slowhometotalFT%','slowhometotalFTA','slowhometotalFTM',
               'slowhometotalOREB','slowhometotalPF','slowhometotal+/-','slowhometotalPPS','slowhometotalSTL',
               'slowhometotalTOV','slowhometotalREB','slowhometotalPOS','slowhometotalOR','slowhometotalDR',
               'slowhometotalSOS','slowhometotalPIE','slowhometotalTIE','fastawaytotalPTS','fastawaytotal3P%','fastawaytotal3PA',
               'fastawaytotal3PM','fastawaytotalAST','fastawaytotalBLK','fastawaytotalDREB','fastawaytotalFG%',
               'fastawaytotalFGA','fastawaytotalFGM','fastawaytotalFT%','fastawaytotalFTA','fastawaytotalFTM',
               'fastawaytotalOREB','fastawaytotalPF','fastawaytotal+/-','fastawaytotalPPS','fastawaytotalSTL',
               'fastawaytotalTOV','fastawaytotalREB','fastawaytotalPOS','fastawaytotalOR','fastawaytotalDR',
               'fastawaytotalSOS','fastawaytotalPIE','fastawaytotalTIE','slowawaytotalPTS','slowawaytotal3P%','slowawaytotal3PA',
               'slowawaytotal3PM','slowawaytotalAST','slowawaytotalBLK','slowawaytotalDREB','slowawaytotalFG%',
               'slowawaytotalFGA','slowawaytotalFGM','slowawaytotalFT%','slowawaytotalFTA','slowawaytotalFTM',
               'slowawaytotalOREB','slowawaytotalPF','slowawaytotal+/-','slowawaytotalPPS','slowawaytotalSTL',
               'slowawaytotalTOV','slowawaytotalREB','slowawaytotalPOS','slowawaytotalOR','slowawaytotalDR',
               'slowawaytotalSOS','slowawaytotalPIE','slowawaytotalTIE','HomeMatchScore','AwayMatchScore', 'MatchScore']

## Viimeisimpien ottelutilastojen lisäys aikaisemmat ottelut sisältävään dataframeen

In [9]:
# Tilastojen haku NBA:n tilastosivulta ja niiden säilöminen omaan dataframeen

path_to_gecko = 'C:/Users/akibr/geckodriver.exe'
browser = webdriver.Firefox(executable_path=path_to_gecko)

url = 'https://stats.nba.com/teams/boxscores/'
browser.get(url)

seasonxpath = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option[2]'

browser.find_element_by_xpath(seasonxpath).click()
time.sleep(10)

#pagexpath = '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]'
#browser.find_element_by_xpath(pagexpath).click()
#time.sleep(15)

table = browser.find_element_by_class_name('nba-stat-table__overflow')

texttable = table.text

games = []

for line_id, lines in enumerate(texttable.split('\n')):
    if line_id == 0:
        column_names = lines.split(' ')
    else:
        games.append(lines)
        
mat = [n.split(' ') for n in games]

column_names[1] = 'TEAM1'
column_names[2] = 'ATVS'
column_names[3] = 'TEAM2'

df_daily = pd.DataFrame(data = mat, columns = column_names)

In [10]:
# Muokataan tilastoista ottelun ajankohta datetime-muotoon ja muut tilastot numeerisiksi

df_daily.drop(labels=['W/L'], axis=1, inplace=True)

for i in range(len(df_daily.columns[4:])):
        
    if i == 0:
        df_daily[df_daily.columns[i+4]] = pd.to_datetime(df_daily[df_daily.columns[i+4]])
        
    else:
        df_daily[df_daily.columns[i+4]] = pd.to_numeric(df_daily[df_daily.columns[i+4]], errors='coerce')
        
df_daily['PPS'] = df_daily.PTS/df_daily.FGA

In [11]:
# Lisätään dataframeen koti ja vierasjoukkueille omat tilastosarakkeet

df_daily['homeTeam'] = ""
df_daily['awayTeam'] = ""

for i, row in df_daily.iterrows():
    if df_daily.at[i,'ATVS'] == '@':
        df_daily.at[i,'homeTeam'] = df_daily.at[i,'TEAM2']
        df_daily.at[i,'awayTeam'] = df_daily.at[i,'TEAM1']
        
    else:
        df_daily.at[i,'homeTeam'] = df_daily.at[i,'TEAM1']
        df_daily.at[i,'awayTeam'] = df_daily.at[i,'TEAM2']


for i in range(len(home_stats1)):
        df_daily.loc[:,home_stats1[i]] = np.nan
        
for i in range(len(away_stats1)):
        df_daily.loc[:,away_stats1[i]] = np.nan
        
for i, row in df_daily.iterrows():
    
    if df_daily.at[i,'homeTeam'] == df_daily.at[i,'TEAM']:
        
        for stat in orig_stats1:
            
            df_daily.at[i, 'home{}'.format(stat)] = df_daily.at[i,'{}'.format(stat)]
            df_daily.at[i, 'home{}'.format(stat)] = df_daily.at[i,'{}'.format(stat)]
            
    elif df_daily.at[i,'awayTeam'] == df_daily.at[i,'TEAM']:
        
        for stat in orig_stats1:
        
            df_daily.at[i, 'away{}'.format(stat)] = df_daily.at[i,'{}'.format(stat)]
            df_daily.at[i, 'away{}'.format(stat)] = df_daily.at[i,'{}'.format(stat)]

In [ ]:
# Jaetaan koti ja vierasjoukkueen tilastot omiin taulukoihin, joista poistetaan virheellinen data
# df_daily_core taulukossa on otteluiden perustiedot, kuten kausi ja ottelupäivämäärä sekä koti- ja vierasjoukkueen nimet. Näistä poistetaan tuplakappaleet

df_daily.drop(labels=['TEAM','TEAM1','ATVS','TEAM2'], axis=1, inplace=True)

df_daily_home = df_daily.iloc[:,[0,22,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43]]
df_daily_away = df_daily.iloc[:,[0,23,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63]]
df_daily_core = df_daily.iloc[:,[0,1,22,23]]
df_daily_home.dropna(inplace=True)
df_daily_away.dropna(inplace=True)
df_daily_core.drop_duplicates(inplace=True)

In [13]:
# Lisätään koti ja vierastilastot muokkauksen jälkeen jälleen yhteen ja otetaan kokonaistaulukosta nykyistä vuotta vastaavat tilastot

df_daily_complete = join_df(df_daily_core, df_daily_home, ['DATE','homeTeam'])
df_daily_complete = join_df(df_daily_complete, df_daily_away, ['DATE', 'awayTeam'])

df_daily_complete['HomeMatchScore'] = df_daily_complete.homePTS
df_daily_complete['AwayMatchScore'] = df_daily_complete.awayPTS
df_daily_complete['MatchScore'] = df_daily_complete.awayPTS+df_daily_complete.homePTS

df_daily_complete = df_daily_complete.iloc[::-1]
df_daily_complete.reset_index(drop=True, inplace=True)

df_daily_complete['Season'] = '2019-2020'

In [14]:
# Lasketaan vielä joitain ekstratilastoja, joita voidaan hyödyntää mallinnuksessa, kuten pallon hallintaa ja reboundien määrää

df_daily_complete['homePOS'] = 0.96*(df_daily_complete.homeFGA+df_daily_complete.homeTOV+(0.475*df_daily_complete.homeFTA)-df_daily_complete.homeOREB)
df_daily_complete['awayPOS'] = 0.96*(df_daily_complete.awayFGA+df_daily_complete.awayTOV+(0.475*df_daily_complete.awayFTA)-df_daily_complete.awayOREB)

df_daily_complete['homeOR'] = 100*(df_daily_complete.homePTS/df_daily_complete.homePOS)
df_daily_complete['homeDR'] = 100*(df_daily_complete.awayPTS/df_daily_complete.homePOS)
df_daily_complete['homeSOS'] = 100*(df_daily_complete.homeOR-df_daily_complete.homeDR)
df_daily_complete['homePIE'] = df_daily_complete.homePTS+df_daily_complete.homeFGM+df_daily_complete.homeFTM-df_daily_complete.homeFGA-df_daily_complete.homeFTA+df_daily_complete.homeDREB+(0.5*df_daily_complete.homeOREB)

df_daily_complete['awayOR'] = 100*(df_daily_complete.awayPTS/df_daily_complete.awayPOS)
df_daily_complete['awayDR'] = 100*(df_daily_complete.homePTS/df_daily_complete.awayPOS)
df_daily_complete['awaySOS'] = 100*(df_daily_complete.awayOR-df_daily_complete.awayDR)
df_daily_complete['awayPIE'] = df_daily_complete.awayPTS+df_daily_complete.awayFGM+df_daily_complete.awayFTM-df_daily_complete.awayFGA-df_daily_complete.awayFTA+df_daily_complete.awayDREB+(0.5*df_daily_complete.awayOREB)

df_daily_complete['homeTIE'] = df_daily_complete.homePIE/(df_daily_complete.homePIE+df_daily_complete.awayPIE)
df_daily_complete['awayTIE'] = df_daily_complete.awayPIE/(df_daily_complete.homePIE+df_daily_complete.awayPIE)

In [15]:
# Muutetaan kaikki pelitilastot per minuutti muotoon

for i, row in df_daily_complete.iterrows():
    for stat in home_stats2[:-1]:
        
        df_daily_complete.at[i,stat] = df_daily_complete.at[i,stat]/(df_daily_complete.at[i,'MIN']/5)
        
    for stat in away_stats2[:-1]:
        
        df_daily_complete.at[i,stat] = df_daily_complete.at[i,stat]/(df_daily_complete.at[i,'MIN']/5)
        
df_daily_complete.drop(labels=['MIN'], axis=1, inplace=True)

In [ ]:
# Lisätään viimeisimmät tilastot kaikkien aikaisemmin kerättyjen tilastojen kanssa samaan taulukkoon, poistetaan tuplat ja tallennetaan taulukko

df_complete = feather.read_dataframe('{}/NBA_complete'.format(PATH))

df_complete = df_complete.append(df_daily_complete, ignore_index=True)

df_complete.drop_duplicates(inplace=True)

df_complete.reset_index(drop=True, inplace=True)

df_complete.to_feather('{}/NBA_complete'.format(PATH))

In [17]:
display_all(df_complete.tail(10))

,AwayMatchScore,DATE,HomeMatchScore,MatchScore,Season,away+/-,away3P%,away3PA,away3PM,awayAST,awayBLK,awayDR,awayDREB,awayFG%,awayFGA,awayFGM,awayFT%,awayFTA,awayFTM,awayOR,awayOREB,awayPF,awayPIE,awayPOS,awayPPS,awayPTS,awayREB,awaySOS,awaySTL,awayTIE,awayTOV,awayTeam,home+/-,home3P%,home3PA,home3PM,homeAST,homeBLK,homeDR,homeDREB,homeFG%,homeFGA,homeFGM,homeFT%,homeFTA,homeFTM,homeOR,homeOREB,homePF,homePIE,homePOS,homePPS,homePTS,homeREB,homeSOS,homeSTL,homeTIE,homeTOV,homeTeam
5898,101.0,2020-03-09,92.0,193.0,2019-2020,0.187500,0.752083,0.750000,0.270833,0.458333,0.041667,2.046671,0.812500,0.904167,1.729167,0.750000,1.852083,0.375000,0.333333,2.246889,0.291667,0.291667,2.041667,1.9510,0.025351,2.104167,1.104167,20.021784,0.187500,0.586826,0.416667,TOR,-0.187500,0.775000,0.895833,0.333333,0.500000,0.083333,2.315732,0.583333,0.829167,1.729167,0.687500,1.487500,0.291667,0.208333,2.109380,0.125000,0.416667,1.437500,1.8930,0.023092,1.916667,0.708333,-20.635235,0.250000,0.413174,0.229167,UTA
5899,114.0,2020-03-10,111.0,225.0,2019-2020,0.062500,0.731250,0.770833,0.270833,0.395833,0.104167,2.351249,0.729167,0.918750,1.937500,0.854167,1.800000,0.458333,0.395833,2.414796,0.229167,0.312500,2.072917,2.0490,0.025538,2.375000,0.958333,6.354726,0.187500,0.496259,0.208333,BOS,-0.062500,0.781250,0.500000,0.187500,0.479167,0.166667,2.468404,0.687500,1.031250,1.937500,0.958333,1.893750,0.229167,0.208333,2.403446,0.208333,0.416667,2.104167,2.0045,0.024866,2.312500,0.895833,-6.495801,0.104167,0.503741,0.250000,IND
5900,115.0,2020-03-10,122.0,237.0,2019-2020,-0.145833,0.793750,0.437500,0.166667,0.625000,0.062500,2.480740,0.583333,1.081250,1.687500,0.875000,1.545833,0.645833,0.479167,2.338402,0.166667,0.604167,2.083333,2.1345,0.029578,2.395833,0.750000,-14.233752,0.208333,0.479616,0.395833,NYK,0.145833,0.833333,0.729167,0.291667,0.479167,0.083333,2.368360,0.583333,0.985417,1.541667,0.729167,1.758333,0.937500,0.791667,2.512521,0.187500,0.520833,2.260417,2.1075,0.034347,2.541667,0.770833,14.416106,0.166667,0.520384,0.395833,WAS
5901,111.0,2020-03-10,117.0,228.0,2019-2020,-0.125000,0.833333,0.729167,0.291667,0.437500,0.000000,2.419883,0.687500,1.004167,1.729167,0.833333,1.539583,0.479167,0.354167,2.295787,0.166667,0.416667,2.062500,2.0985,0.027861,2.312500,0.854167,-12.409658,0.187500,0.491315,0.395833,MIN,0.125000,0.822917,0.791667,0.312500,0.500000,0.104167,2.286525,0.583333,1.068750,1.666667,0.854167,1.602083,0.541667,0.416667,2.410121,0.104167,0.416667,2.135417,2.1070,0.030469,2.437500,0.687500,12.359595,0.187500,0.508685,0.375000,HOU
5902,103.0,2020-03-10,108.0,211.0,2019-2020,-0.104167,0.520833,0.500000,0.125000,0.604167,0.104167,2.345509,0.541667,1.041667,1.666667,0.833333,1.539583,0.479167,0.354167,2.236921,0.187500,0.312500,1.822917,1.9985,0.026823,2.145833,0.729167,-10.858839,0.187500,0.461741,0.375000,CLE,0.104167,0.735417,0.708333,0.250000,0.479167,0.125000,2.277955,0.666667,1.041667,1.750000,0.875000,1.666667,0.312500,0.250000,2.388535,0.291667,0.437500,2.125000,1.9625,0.026786,2.250000,0.958333,11.058033,0.166667,0.538259,0.437500,CHI
5903,120.0,2020-03-10,115.0,235.0,2019-2020,0.104167,0.883333,0.687500,0.291667,0.645833,0.062500,2.344443,0.604167,0.972917,1.916667,0.895833,1.893750,0.458333,0.416667,2.446375,0.250000,0.541667,2.166667,2.1290,0.027174,2.500000,0.854167,10.193231,0.208333,0.503632,0.333333,ORL,-0.104167,0.789583,0.604167,0.229167,0.500000,0.104167,2.410150,0.666667,1.041667,1.583333,0.791667,1.535417,0.791667,0.583333,2.309727,0.145833,0.458333,2.135417,2.1610,0.031524,2.395833,0.812500,-10.042290,0.145833,0.496368,0.437500,MEM
5904,109.0,2020-03-10,119.0,228.0,2019-2020,-0.208333,0.804167,0.916667,0.354167,0.479167,0.062500,2.477779,0.729167,0.952083,1.916667,0.875000,1.514583,0.229167,0.166667,2.269562,0.145833,0.395833,1.968750,2.0845,0.024683,2.270833,0.875000,-20.821673,0.166667,0.462103,0.291667,DAL,0.208333,0.977083,0.666667,0.312500,0.666667,0.208333,2.316231,0.708333,1.077083,1.854167,0.958333,1.666667,0.3125

## Liikkuvien keskiarvojen laskeminen tilastoista

Seuraavien funktioiden avulla lasketaan kaikista tilastoista kaksi eri liikkuvaa keskiarvoa ja lopullisessa **df_final** dataframesta luokitellaan kategoria ja jatkuvat muuttujat erikseen. Tätä lopullista taulukkodataa käytetään Deep Learning -mallin opettamiseen.

In [19]:
#df_complete = feather.read_dataframe('{}/NBA_complete'.format(PATH))

In [20]:
teams = df_complete.homeTeam.unique()
teams.sort()

seasons = df_complete.Season.unique()

In [21]:
def calculate_teamavgs(df, moving_averages): 
    
    for season in seasons:
        
        df_season = df[df.loc[:,('Season')] == season]
        
        for team in teams:
            
            df_hometeam = df_season[df_season.loc[:,('homeTeam')] == team]
            df_hometeam = df_hometeam.loc[:,home_stats2]
        
            df_awayteam = df_season[df_season.loc[:,('awayTeam')] == team]
            df_awayteam = df_awayteam.loc[:,away_stats2]
            
        
            for stat in df_hometeam.columns:
            
                df_hometeam.loc[:,'fast{}'.format(stat)] = df_hometeam[stat].rolling(window=moving_averages[0]).apply(weighted_ma(fast_weights)).shift(1)
                df_hometeam.loc[:,'slow{}'.format(stat)] = df_hometeam[stat].rolling(window=moving_averages[1]).apply(weighted_ma(slow_weights)).shift(1)
            
            for stat in df_awayteam.columns:
        
                df_awayteam.loc[:,'fast{}'.format(stat)] = df_awayteam[stat].rolling(window=moving_averages[0]).apply(weighted_ma(fast_weights)).shift(1)
                df_awayteam.loc[:,'slow{}'.format(stat)] = df_awayteam[stat].rolling(window=moving_averages[1]).apply(weighted_ma(slow_weights)).shift(1)
        
            df_hometeam.drop(columns=home_stats2, inplace=True)
            df_awayteam.drop(columns=away_stats2, inplace=True)
            
        
            df = df.combine_first(df_hometeam)
            df = df.combine_first(df_awayteam)
    
    
    return df    

In [22]:
def calculate_totalavgs(df, moving_averages):
    
    for season in seasons:
        
        df_season = df[df.loc[:,('Season')] == season]
        
        for team in teams:
            
            df_hometeam = df_season[df_season.loc[:,('homeTeam')] == team]
            df_awayteam = df_season[df_season.loc[:,('awayTeam')] == team]
            
            df_team = df_hometeam.append(df_awayteam, ignore_index=False)
            df_team = df_team.sort_index()
            
            for i in range(len(total_stats)):
                df_team.loc[:,total_stats[i]] = np.nan
            
            for i, row in df_team.iterrows():
                
                for stat in orig_stats2:
                    
                    if df_team.at[i,'homeTeam'] == team:
                        df_team.at[i, 'total{}'.format(stat)] = df_team.at[i, 'home{}'.format(stat)]
        
                    elif df_team.at[i,'awayTeam'] == team:
                        df_team.at[i, 'total{}'.format(stat)] = df_team.at[i, 'away{}'.format(stat)]
                    
            df_team = df_team.loc[:,total_stats]
        
            for stat in df_team.columns:
                
                df_team.loc[:,'fast{}'.format(stat)] = df_team[stat].rolling(window=moving_averages[0]).apply(weighted_ma(fast_weights)).shift(1)
                df_team.loc[:,'slow{}'.format(stat)] = df_team[stat].rolling(window=moving_averages[1]).apply(weighted_ma(slow_weights)).shift(1)
            
            df_team.drop(columns = total_stats, inplace=True)
            
            for i, row in df_team.iterrows():
                
                for stat in orig_stats2:
                    
                    if df.at[i,'homeTeam'] == team:
        
                        df.at[i, 'fasthometotal{}'.format(stat)] = df_team.at[i,'fasttotal{}'.format(stat)]
                        df.at[i, 'slowhometotal{}'.format(stat)] = df_team.at[i,'slowtotal{}'.format(stat)]
            
                    elif df.at[i,'awayTeam'] == team:
        
                        df.at[i, 'fastawaytotal{}'.format(stat)] = df_team.at[i,'fasttotal{}'.format(stat)]
                        df.at[i, 'slowawaytotal{}'.format(stat)] = df_team.at[i,'slowtotal{}'.format(stat)]
    
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [ ]:
df_final = calculate_teamavgs(df_complete, moving_averages)
df_final = calculate_totalavgs(df_final, moving_averages)

In [24]:
for v in cat_vars: df_final[v] = df_final[v].astype('category').cat.as_ordered()

for v in contin_vars:
    df_final[v] = df_final[v].fillna(0).astype('float32')

df_final = df_final[cat_vars+contin_vars]

In [25]:
df_final.to_feather('{}/NBA_final'.format(PATH))

In [26]:
display_all(df_final.tail(10))
df_final.shape

,Season,homeTeam,awayTeam,fasthomePTS,fasthome3P%,fasthome3PA,fasthome3PM,fasthomeAST,fasthomeBLK,fasthomeDREB,fasthomeFG%,fasthomeFGA,fasthomeFGM,fasthomeFT%,fasthomeFTA,fasthomeFTM,fasthomeOREB,fasthomePF,fasthome+/-,fasthomePPS,fasthomeSTL,fasthomeTOV,fasthomeREB,fasthomePOS,fasthomeOR,fasthomeDR,fasthomeSOS,fasthomePIE,fasthomeTIE,slowhomePTS,slowhome3P%,slowhome3PA,slowhome3PM,slowhomeAST,slowhomeBLK,slowhomeDREB,slowhomeFG%,slowhomeFGA,slowhomeFGM,slowhomeFT%,slowhomeFTA,slowhomeFTM,slowhomeOREB,slowhomePF,slowhome+/-,slowhomePPS,slowhomeSTL,slowhomeTOV,slowhomeREB,slowhomePOS,slowhomeOR,slowhomeDR,slowhomeSOS,slowhomePIE,slowhomeTIE,fastawayPTS,fastaway3P%,fastaway3PA,fastaway3PM,fastawayAST,fastawayBLK,fastawayDREB,fastawayFG%,fastawayFGA,fastawayFGM,fastawayFT%,fastawayFTA,fastawayFTM,fastawayOREB,fastawayPF,fastaway+/-,fastawayPPS,fastawaySTL,fastawayTOV,fastawayREB,fastawayPOS,fastawayOR,fastawayDR,fastawaySOS,fastawayPIE,fastawayTIE,slowawayPTS,slowaway3P%,slowaway3PA,slowaway3PM,slowawayAST,slowawayBLK,slowawayDREB,slowawayFG%,slowawayFGA,slowawayFGM,slowawayFT%,slowawayFTA,slowawayFTM,slowawayOREB,slowawayPF,slowaway+/-,slowawayPPS,slowawaySTL,slowawayTOV,slowawayREB,slowawayPOS,slowawayOR,slowawayDR,slowawaySOS,slowawayPIE,slowawayTIE,fasthometotalPTS,fasthometotal3P%,fasthometotal3PA,fasthometotal3PM,fasthometotalAST,fasthometotalBLK,fasthometotalDREB,fasthometotalFG%,fasthometotalFGA,fasthometotalFGM,fasthometotalFT%,fasthometotalFTA,fasthometotalFTM,fasthometotalOREB,fasthometotalPF,fasthometotal+/-,fasthometotalPPS,fasthometotalSTL,fasthometotalTOV,fasthometotalREB,fasthometotalPOS,fasthometotalOR,fasthometotalDR,fasthometotalSOS,fasthometotalPIE,fasthometotalTIE,slowhometotalPTS,slowhometotal3P%,slowhometotal3PA,slowhometotal3PM,slowhometotalAST,slowhometotalBLK,slowhometotalDREB,slowhometotalFG%,slowhometotalFGA,slowhometotalFGM,slowhometotalFT%,slowhometotalFTA,slowhometotalFTM,slowhometotalOREB,slowhometotalPF,slowhometotal+/-,slowhometotalPPS,slowhometotalSTL,slowhometotalTOV,slowhometotalREB,slowhometotalPOS,slowhometotalOR,slowhometotalDR,slowhometotalSOS,slowhometotalPIE,slowhometotalTIE,fastawaytotalPTS,fastawaytotal3P%,fastawaytotal3PA,fastawaytotal3PM,fastawaytotalAST,fastawaytotalBLK,fastawaytotalDREB,fastawaytotalFG%,fastawaytotalFGA,fastawaytotalFGM,fastawaytotalFT%,fastawaytotalFTA,fastawaytotalFTM,fastawaytotalOREB,fastawaytotalPF,fastawaytotal+/-,fastawaytotalPPS,fastawaytotalSTL,fastawaytotalTOV,fastawaytotalREB,fastawaytotalPOS,fastawaytotalOR,fastawaytotalDR,fastawaytotalSOS,fastawaytotalPIE,fastawaytotalTIE,slowawaytotalPTS,slowawaytotal3P%,slowawaytotal3PA,slowawaytotal3PM,slowawaytotalAST,slowawaytotalBLK,slowawaytotalDREB,slowawaytotalFG%,slowawaytotalFGA,slowawaytotalFGM,slowawaytotalFT%,slowawaytotalFTA,slowawaytotalFTM,slowawaytotalOREB,slowawaytotalPF,slowawaytotal+/-,slowawaytotalPPS,slowawaytotalSTL,slowawaytotalTOV,slowawaytotalREB,slowawaytotalPOS,slowawaytotalOR,slowawaytotalDR,slowawaytotalSOS,slowawaytotalPIE,slowawaytotalTIE,HomeMatchScore,AwayMatchScore,MatchScore
4900,2019-2020,UTA,TOR,2.430460,0.868198,0.739536,0.307268,0.399924,0.076865,0.673992,0.988480,1.756279,0.834094,1.586282,0.585712,0.455004,0.252568,0.423040,-0.071347,0.029104,0.137177,0.302511,0.926560,2.001057,2.526390,2.606246,-7.985608,2.177844,0.494162,2.391952,0.809587,0.724958,0.282875,0.411416,0.075868,0.719959,0.979488,1.767042,0.831446,1.621025,0.566032,0.446184,0.224750,0.398672,-0.074870,0.028422,0.118517,0.303605,0.944709,2.030171,2.455392,2.534904,-7.951146,2.168843,0.497282,2.492295,0.735569,0.920757,0.327911,0.573820,0.062500,0.730974,0.994463,1.726313,0.823820,1.762338,0.607782,0.516743,0.052987,0.480974,0.138128,0.030077,0.208333,0.226123,0.783961,2.100619,2.472864,2.336633,13.623025,2.256231,0.519324,2.463625,0.728487,0.884022,0.310661,0.534388,0.070250,0.727087,0.976859,1.752868,0.820808,1.757756,0.602388,0.511349,0.082069,0.483501,0.087301,0.029329,0.199441,0.240700,0.809156,2.109728,2.4330

(4910, 214)

## Deep Learning -mallin ennustukseen käyttämän taulukon kokoaminen

In [27]:
# Kerätään tulevat ottelut omaan taulukkoon

df_matchup = pd.read_excel(f'{PATH}Matchup.xlsx', parse_dates = [1])

df_matchup = df_matchup[df_matchup.loc[:,('GameDate')] == bet_date]
df_matchup['homeTeam'] = df_matchup['homeTeam'].str.strip()
df_matchup['awayTeam'] = df_matchup['awayTeam'].str.strip()
df_matchup['awayTeam'] = df_matchup['awayTeam'].map(team_map)
df_matchup['homeTeam'] = df_matchup['homeTeam'].map(team_map)
df_matchup.reset_index(drop=True, inplace=True)

In [28]:
display(df_matchup)

,Season,GameDate,awayTeam,homeTeam
0,2019-2020,2020-03-11,CHA,MIA
1,2019-2020,2020-03-11,DET,PHI
2,2019-2020,2020-03-11,NYK,ATL
3,2019-2020,2020-03-11,UTA,OKC
4,2019-2020,2020-03-11,DEN,DAL
5,2019-2020,2020-03-11,NOP,SAC


In [29]:
def calculate_predteamavgs(df, moving_averages): 
    
    for i, row in df_matchup.iterrows():
        home_team = df_matchup.at[i,'homeTeam']
        away_team = df_matchup.at[i,'awayTeam']
        
            
        df_hometeam = df[df.loc[:,('homeTeam')] == home_team]
        df_hometeam = df_hometeam.loc[:,home_stats2]
        
        df_awayteam = df[df.loc[:,('awayTeam')] == away_team]
        df_awayteam = df_awayteam.loc[:,away_stats2]
            
        
        for stat in df_hometeam.columns:
            
            df_hometeam.loc[:,'fast{}'.format(stat)] = df_hometeam[stat].rolling(window=moving_averages[0]).apply(weighted_ma(fast_weights)).shift(1)
            df_hometeam.loc[:,'slow{}'.format(stat)] = df_hometeam[stat].rolling(window=moving_averages[1]).apply(weighted_ma(slow_weights)).shift(1)
            
        for stat in df_awayteam.columns:
        
            df_awayteam.loc[:,'fast{}'.format(stat)] = df_awayteam[stat].rolling(window=moving_averages[0]).apply(weighted_ma(fast_weights)).shift(1)
            df_awayteam.loc[:,'slow{}'.format(stat)] = df_awayteam[stat].rolling(window=moving_averages[1]).apply(weighted_ma(slow_weights)).shift(1)
        
        df_hometeam.drop(columns=home_stats2, inplace=True)
        df_awayteam.drop(columns=away_stats2, inplace=True)
            
        
        df = df.combine_first(df_hometeam)
        df = df.combine_first(df_awayteam)

    return df    

In [30]:
def calculate_predtotalavgs(df, moving_averages):
    
    for i, row in df_matchup.iterrows():
        
        team1 = df_matchup.at[i,'homeTeam']
        team2 = df_matchup.at[i,'awayTeam']
            
        df_hometeam1 = df[df.loc[:,('homeTeam')] == team1]
        df_awayteam1 = df[df.loc[:,('awayTeam')] == team1]
            
        df_team1 = df_hometeam1.append(df_awayteam1, ignore_index=False)
        df_team1 = df_team1.sort_index()
        
        df_hometeam2 = df[df.loc[:,('homeTeam')] == team2]
        df_awayteam2 = df[df.loc[:,('awayTeam')] == team2]
            
        df_team2 = df_hometeam2.append(df_awayteam2, ignore_index=False)
        df_team2 = df_team2.sort_index()
        
        teams = {team1: df_team1, team2: df_team2}
        
        for team in teams:
            
            df_team = teams[team]
            
            for i in range(len(total_stats)):
                df_team.loc[:,total_stats[i]] = np.nan
            
            for i, row in df_team.iterrows():
                
                for stat in orig_stats2:
                    
                    if df_team.at[i,'homeTeam'] == team:
                        df_team.at[i, 'total{}'.format(stat)] = df_team.at[i, 'home{}'.format(stat)]
        
                    elif df_team.at[i,'awayTeam'] == team:
                        df_team.at[i, 'total{}'.format(stat)] = df_team.at[i, 'away{}'.format(stat)]
                    
            df_team = df_team.loc[:,total_stats]
            
            for stat in df_team.columns:
                
                df_team.loc[:,'fast{}'.format(stat)] = df_team[stat].rolling(window=moving_averages[0]).apply(weighted_ma(fast_weights)).shift(1)
                df_team.loc[:,'slow{}'.format(stat)] = df_team[stat].rolling(window=moving_averages[1]).apply(weighted_ma(slow_weights)).shift(1)
            
            df_team.drop(columns = total_stats, inplace=True)
            
            for i, row in df_team.iterrows():
                
                for stat in orig_stats2:
                    
                    if df.at[i,'homeTeam'] == team:
        
                        df.at[i, 'fasthometotal{}'.format(stat)] = df_team.at[i,'fasttotal{}'.format(stat)]
                        df.at[i, 'slowhometotal{}'.format(stat)] = df_team.at[i,'slowtotal{}'.format(stat)]
            
                    elif df.at[i,'awayTeam'] == team:
        
                        df.at[i, 'fastawaytotal{}'.format(stat)] = df_team.at[i,'fasttotal{}'.format(stat)]
                        df.at[i, 'slowawaytotal{}'.format(stat)] = df_team.at[i,'slowtotal{}'.format(stat)]
    
    
    return df

Samalla tavalla kuin yllä laskettiin liikkuvat keskiarvot kaikille tilastoille, tässä se tehdään vain nykyisen kauden tähän astisille tilastoille, joita käytetään yllä kerättyjen tulevien otteluiden tulosten ennustamisessa. Tulokset asetetaan nollaksi, jotka korvataan myöhemmin DL-mallin ennustuksilla

In [ ]:
df_predict = df_complete[df_complete.loc[:,('Season')] == '2019-2020']
df_predict = df_predict.append(df_matchup, ignore_index=True, sort=True)

df_predict = calculate_predteamavgs(df_predict, moving_averages)
df_predict = calculate_predtotalavgs(df_predict, moving_averages)

df_predict = df_predict[df_predict.loc[:,('GameDate')] == bet_date]
df_predict = df_predict.dropna(axis=1)
df_predict.drop(labels='GameDate',axis=1,inplace=True)
df_predict['HomeMatchScore'] = ""
df_predict.loc[:,'HomeMatchScore'] = 0
df_predict['AwayMatchScore'] = ""
df_predict.loc[:,'AwayMatchScore'] = 0
df_predict['MatchScore'] = ""
df_predict.loc[:,'MatchScore'] = 0
df_predict.reset_index(drop=True, inplace=True)

for v in cat_vars: df_predict[v] = df_predict[v].astype('category').cat.as_ordered()
    
for v in contin_vars:
    df_predict[v] = df_predict[v].fillna(0).astype('float32')
    
df_predict = df_predict[cat_vars+contin_vars]
df_predict.reset_index(drop=True, inplace=True)

In [32]:
display_all(df_predict)

,Season,homeTeam,awayTeam,fasthomePTS,fasthome3P%,fasthome3PA,fasthome3PM,fasthomeAST,fasthomeBLK,fasthomeDREB,fasthomeFG%,fasthomeFGA,fasthomeFGM,fasthomeFT%,fasthomeFTA,fasthomeFTM,fasthomeOREB,fasthomePF,fasthome+/-,fasthomePPS,fasthomeSTL,fasthomeTOV,fasthomeREB,fasthomePOS,fasthomeOR,fasthomeDR,fasthomeSOS,fasthomePIE,fasthomeTIE,slowhomePTS,slowhome3P%,slowhome3PA,slowhome3PM,slowhomeAST,slowhomeBLK,slowhomeDREB,slowhomeFG%,slowhomeFGA,slowhomeFGM,slowhomeFT%,slowhomeFTA,slowhomeFTM,slowhomeOREB,slowhomePF,slowhome+/-,slowhomePPS,slowhomeSTL,slowhomeTOV,slowhomeREB,slowhomePOS,slowhomeOR,slowhomeDR,slowhomeSOS,slowhomePIE,slowhomeTIE,fastawayPTS,fastaway3P%,fastaway3PA,fastaway3PM,fastawayAST,fastawayBLK,fastawayDREB,fastawayFG%,fastawayFGA,fastawayFGM,fastawayFT%,fastawayFTA,fastawayFTM,fastawayOREB,fastawayPF,fastaway+/-,fastawayPPS,fastawaySTL,fastawayTOV,fastawayREB,fastawayPOS,fastawayOR,fastawayDR,fastawaySOS,fastawayPIE,fastawayTIE,slowawayPTS,slowaway3P%,slowaway3PA,slowaway3PM,slowawayAST,slowawayBLK,slowawayDREB,slowawayFG%,slowawayFGA,slowawayFGM,slowawayFT%,slowawayFTA,slowawayFTM,slowawayOREB,slowawayPF,slowaway+/-,slowawayPPS,slowawaySTL,slowawayTOV,slowawayREB,slowawayPOS,slowawayOR,slowawayDR,slowawaySOS,slowawayPIE,slowawayTIE,fasthometotalPTS,fasthometotal3P%,fasthometotal3PA,fasthometotal3PM,fasthometotalAST,fasthometotalBLK,fasthometotalDREB,fasthometotalFG%,fasthometotalFGA,fasthometotalFGM,fasthometotalFT%,fasthometotalFTA,fasthometotalFTM,fasthometotalOREB,fasthometotalPF,fasthometotal+/-,fasthometotalPPS,fasthometotalSTL,fasthometotalTOV,fasthometotalREB,fasthometotalPOS,fasthometotalOR,fasthometotalDR,fasthometotalSOS,fasthometotalPIE,fasthometotalTIE,slowhometotalPTS,slowhometotal3P%,slowhometotal3PA,slowhometotal3PM,slowhometotalAST,slowhometotalBLK,slowhometotalDREB,slowhometotalFG%,slowhometotalFGA,slowhometotalFGM,slowhometotalFT%,slowhometotalFTA,slowhometotalFTM,slowhometotalOREB,slowhometotalPF,slowhometotal+/-,slowhometotalPPS,slowhometotalSTL,slowhometotalTOV,slowhometotalREB,slowhometotalPOS,slowhometotalOR,slowhometotalDR,slowhometotalSOS,slowhometotalPIE,slowhometotalTIE,fastawaytotalPTS,fastawaytotal3P%,fastawaytotal3PA,fastawaytotal3PM,fastawaytotalAST,fastawaytotalBLK,fastawaytotalDREB,fastawaytotalFG%,fastawaytotalFGA,fastawaytotalFGM,fastawaytotalFT%,fastawaytotalFTA,fastawaytotalFTM,fastawaytotalOREB,fastawaytotalPF,fastawaytotal+/-,fastawaytotalPPS,fastawaytotalSTL,fastawaytotalTOV,fastawaytotalREB,fastawaytotalPOS,fastawaytotalOR,fastawaytotalDR,fastawaytotalSOS,fastawaytotalPIE,fastawaytotalTIE,slowawaytotalPTS,slowawaytotal3P%,slowawaytotal3PA,slowawaytotal3PM,slowawaytotalAST,slowawaytotalBLK,slowawaytotalDREB,slowawaytotalFG%,slowawaytotalFGA,slowawaytotalFGM,slowawaytotalFT%,slowawaytotalFTA,slowawaytotalFTM,slowawaytotalOREB,slowawaytotalPF,slowawaytotal+/-,slowawaytotalPPS,slowawaytotalSTL,slowawaytotalTOV,slowawaytotalREB,slowawaytotalPOS,slowawaytotalOR,slowawaytotalDR,slowawaytotalSOS,slowawaytotalPIE,slowawaytotalTIE,HomeMatchScore,AwayMatchScore,MatchScore
0,2019-2020,MIA,CHA,2.343417,0.970814,0.842085,0.394406,0.607211,0.092846,0.714897,0.990287,1.748383,0.827530,1.630108,0.374429,0.293950,0.143645,0.386130,0.149068,0.028041,0.121385,0.232401,0.858543,1.934393,2.526923,2.368067,15.885530,2.128805,0.523376,2.412952,0.969437,0.803892,0.376306,0.616516,0.092360,0.704207,1.020781,1.720491,0.839095,1.615733,0.463206,0.358455,0.139904,0.398059,0.147950,0.029370,0.139315,0.256235,0.844111,1.974572,2.548521,2.391494,15.702757,2.200964,0.524745,2.118597,0.741338,0.731489,0.282764,0.499764,0.111830,0.646735,0.816365,1.771680,0.756675,1.405200,0.431454,0.322482,0.196829,0.380298,-0.201179,0.022908,0.137446,0.210502,0.843565,1.910681,2.110202,2.315579,-20.537737,1.739770,0.455068,2.083958,0.723307,0.712941,0.265115,0.489183,0.097263,0.651874,0.824083,1.782955,0.749455,1.378410,0.441696,0.319933,0.207736,0.361014,-0.219212,0.022981,0.150278,0.228316,0.859611,1.932807,2.099112

In [33]:
df_predict.to_feather('{}/NBA_predict'.format(PATH))

## Deep Learning malli O/U tuloksien ennustamiseen

Aikaisemmin kootusta df_final taulukosta erotellaan erikeen koti, vieras ja koko ottelun tilastotaulukot. Sama tehdään myös ennustettavat ottelut sisältävälle taulukolle. Koti- ja vierasjoukkueiden pisteille sekä kokonaisottelun tulokselle tehdään kaikille oma DL-malli, joita harjoitetaan df_home, df_away ja df_match taulukoissa olevalla datalla pari epochia. Kyseisillä malleilla ennustetaan predict-taulukoissa olevien otteluiden tulokset niihin lasketuiden tilastojen perusteella.

In [ ]:
df_home = df_final.drop(labels=['AwayMatchScore','MatchScore'],axis=1)
df_away = df_final.drop(labels=['HomeMatchScore','MatchScore'],axis=1)
df_match = df_final.drop(labels=['HomeMatchScore','AwayMatchScore'],axis=1)

df_predict_home = df_predict.drop(labels='AwayMatchScore',axis=1)
df_predict_away = df_predict.drop(labels='HomeMatchScore',axis=1)
df_predict_match = df_predict.drop(labels=['HomeMatchScore','AwayMatchScore'],axis=1)

In [ ]:
# Eri riippuvien muuttujien määrittäminen sekä DL-mallien asetukset

dep_home = 'HomeMatchScore'
dep_away = 'AwayMatchScore'
dep_match = 'MatchScore'
cat_list = cat_vars
contin_list = contin_vars[:-3]
procs = [FillMissing, Categorify, Normalize]

In [ ]:
home_data = (TabularList.from_df(df_home, path=PATH, cat_names=cat_list, cont_names=contin_list, procs=procs)
                           .split_by_idx(list(range((len(df_home)-200),(len(df_home)))))
                           .label_from_df(cols=dep_home)
                           .databunch())

In [ ]:
away_data = (TabularList.from_df(df_away, path=PATH, cat_names=cat_list, cont_names=contin_list, procs=procs)
                           .split_by_idx(list(range((len(df_away)-200),(len(df_away)))))
                           .label_from_df(cols=dep_away)
                           .databunch())

In [ ]:
match_data = (TabularList.from_df(df_match, path=PATH, cat_names=cat_list, cont_names=contin_list, procs=procs)
                           .split_by_idx(list(range((len(df_match)-200),(len(df_match)))))
                           .label_from_df(cols=dep_match)
                           .databunch())

In [ ]:
learn_home = tabular_learner(home_data, layers=[200,100,100], metrics=mean_absolute_error)
learn_away = tabular_learner(away_data, layers=[200,100,100], metrics=mean_absolute_error)
learn_match = tabular_learner(match_data, layers=[200,100,100], metrics=mean_absolute_error)

In [ ]:
learn_home.fit(2, 1e-2)

In [ ]:
learn_away.fit(2, 1e-2)

In [ ]:
learn_match.fit(2, 1e-2)

In [ ]:
df_matchup['TeamScore'] = ""
df_matchup['MatchScore'] = ""

for i, row in df_predict_away.iterrows():
    datapred_away = df_predict_away.iloc[i]
    pred_away = learn_away.predict(datapred_away)
    number = float(pred_away[1])
    df_matchup.at[i,'AwayScore'] = number

for i, row in df_predict_home.iterrows():
    datapred_home = df_predict_home.iloc[i]
    pred_home = learn_home.predict(datapred_home)
    number = float(pred_home[1])
    df_matchup.at[i,'HomeScore'] = number
    
for i, row in df_predict_match.iterrows():
    datapred_match = df_predict_match.iloc[i]
    pred_match = learn_match.predict(datapred_match)
    number = float(pred_match[1])
    df_matchup.at[i,'MatchScore'] = number

    
for i, row in df_matchup.iterrows():
    df_matchup.at[i,'TeamScore'] = df_matchup.at[i,'HomeScore'] + df_matchup.at[i,'AwayScore']
    
    
display(df_matchup)

Match-malli laskee ottelun kokonaispistemäärän aikaisempien otteluiden kokonaispistemääristä ja muista tilastoista. Home ja Away -mallit tekevät saman koti ja vierasjoukkueen tilastoista ja näiden kahden mallin ulostulot lasketaan yhteen kokonaispistemäärien saamiseksi. Näiden kahden mallin tulosta voidaan verrata toisiinsa. Tulos näyttää esimerkiksi tältä:

![alt text](https://github.com/SDCH-analytics/NBAOverUnder/blob/main/esimerkki.PNG)